In [ ]:
import numpy as np
from TMDP import TMDP
from model_functions import *
from policy_utils import *
import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *
from algorithms import *
import pandas as pd
import constants
import numpy as np
#np.set_printoptions(precision=4)
import math
from policy_utils import *

import torch
import mlflow
from CurriculumMPI import CurriculumMPI
import optuna

nrows = 20
nS = nrows**2
nA = 4
seed = constants.SEEDS[0]
set_policy_seed(seed)
num_bins = 10
gamma = .999999999
tau = .6
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)
env = FrozenLakeEnv(is_slippery=False, seed=seed, 
                    desc=generate_random_map(nrows, seed=seed), 
                    reward_shape=True,
                    num_bins=num_bins#, render_mode="human"
                    )
xi = np.ones(env.nS) * 1/env.nS
tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=seed)

In [ ]:
res = bellman_optimal_q(tmdp.env.P_mat, tmdp.env.reward, tmdp.gamma)
Q = res["Q"]
V = compute_V_from_Q(Q, get_policy(Q))

In [ ]:

def get_or_create_experiment(experiment_name):
    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)


In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
experiment_name = "Frozen_Lake_{}_{}_{}".format(nS, nA, tau)
experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

In [ ]:
episodes = 1200000
final_temp=1
lam = 1
batch_size = 32
check_convergence=False
param_decay=True
biased=False

In [ ]:

def objective(trial):
    with mlflow.start_run(nested=True):
        global episodes
        # Parameter space optimization
        model_lr = trial.suggest_float("model_lr", 1e-4, 0.5, log=False)
        pol_lr = trial.suggest_float("pol_lr", 1e-4, 0.5, log=False)
        #batch_size = trial.suggest_int("batch_size", 1, 32, log=False)
        #lam = trial.suggest_float("lam", 0., 1., log=False)
        temp = trial.suggest_float("temp", 0.1, 10, log=False)

        # Curriculum class initialization
        curr_MPI = CurriculumMPI(tmdp)
        
        tmdp.update_tau(tau) # Reset the teleport rate 
        
        curr_MPI.train(model_lr, pol_lr, batch_size=batch_size, 
                        lam=lam, temp=temp, final_temp=final_temp,
                        episodes=episodes, check_convergence=check_convergence,
                        param_decay=param_decay, biased=biased)
        
        
        avg_reward = np.average(curr_MPI.reward_records[-10:])/batch_size
        
        mlflow.log_params({
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "batch_size": batch_size,
            "lam": lam,
            "temp": temp,
            "final_temp": final_temp,
            "episodes": curr_MPI.episodes,
        })


        mlflow.log_metric("Avg Reward", avg_reward)

        mlflow.set_tags(tags={
        "Project": "Curriculum Learning",
        "Algorithm": "CurriculumMPI",
        "Environment": tmdp.env.__class__.__name__,
        "nS": nS,
        "nA": nA,
        "gamma": gamma,
        "mu": tmdp.env.mu,
        "xi": tmdp.xi,
        "seed": seed,
        "tau": tau,
        "episodes": curr_MPI.episodes,
        })
        curr_MPI.save_to_mlflow() # Save the model to MLflow

    return avg_reward


In [ ]:

run_name = "CMPI Avg J"
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    
    mlflow.log_params(study.best_params)
    
    # Train and save the best model
    curr_MPI = CurriculumMPI(tmdp)
    tmdp.update_tau(tau) # Reset the teleport rate 

    # Get the best parameters
    model_lr = study.best_params["model_lr"]
    pol_lr = study.best_params["pol_lr"]
    #batch_size = study.best_params["batch_size"]
    #lam = study.best_params["lam"]
    temp = study.best_params["temp"]
    
    # Train the best model
    curr_MPI.train(model_lr, pol_lr, batch_size=batch_size, 
                        lam=lam, temp=temp, final_temp=final_temp,
                        episodes=episodes, check_convergence=check_convergence,
                        param_decay=param_decay, biased=biased)
    
    mlflow.set_tags(tags={
        "Project": "Curriculum Learning",
        "Algorithm": "CurriculumMPI",
        "Environment": tmdp.env.__class__.__name__,
        "nS": nS,
        "nA": nA,
        "gamma": gamma,
        "mu": tmdp.env.mu,
        "xi": tmdp.xi,
        "seed": seed,
        "tau": tau,
        "episodes": episodes,
        })

    curr_MPI.save_to_mlflow() # Save the best model to MLflow

In [ ]:
delta_pol = get_policy(Q) - get_policy(get_softmax_policy(curr_MPI.theta))


V_Q = compute_V_from_Q(Q, get_policy(curr_MPI.Q))
V_curr = compute_V_from_Q(Q, get_softmax_policy(curr_MPI.theta))
V_ref = compute_V_from_Q(Q, get_softmax_policy(curr_MPI.theta_ref))

j_curr = compute_expected_j(V_curr, tmdp.env.mu)
j_curr_ref = compute_expected_j(V_ref, tmdp.env.mu)
j_q_curr = compute_expected_j(V_Q, tmdp.env.mu)
j_opt = compute_expected_j(V, tmdp.env.mu)

print("optimal performance: ",j_opt)
print("curriculum performance: ", j_curr)
print("reference curriculum performance: ",j_curr_ref)
print("Q curriculum performance: ",j_q_curr)


In [ ]:
import matplotlib.pyplot as plt
reward_records = curr_MPI.reward_records
# Generate recent 50 interval average
average_reward = []
scaled_reward = []
for idx in range(len(reward_records)):
    avg_list = np.empty(shape=(1,), dtype=int)
    scaled_reward.append(reward_records[idx]/batch_size)
    if idx < 50:
        avg_list = reward_records[:idx+1]
    else:
        avg_list = reward_records[idx-49:idx+1]
    average_reward.append(np.average(avg_list)/batch_size)
# Plot
#plt.plot(scaled_reward)
plt.plot(average_reward[:-1])

In [ ]:
pi = get_policy(get_softmax_policy(curr_MPI.theta))
tmdp.update_tau(0.)
tmdp.reset()
done = False
step = 0
env.render_mode = "human"
while True:
    s = tmdp.env.s
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    print("state {} action {} next state {} reward {}".format(s, a, s_prime, reward))
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break
